#### Лабораторная работа
Первая краевая задача для стационарного уравнения теплопроводности с кусочно-непрерывными коэффициентами

Задача заключается в нахождении решения обыкновенного дифференциального уравнения второго порядка с кусочно-непрерывными коэффициентами, двумя краевыми условиями первого рода на интервале (0, 1):

\begin{equation}
    \frac{d}{dx}\left[k(x)\frac{du}{dx}\right]-q(x)u=-f(x)
\end{equation}
\begin{equation}
    u(0) = u^0, u(1) = u^1
\end{equation}

Решение задачи состоит из нескольких этапов:

    1. Аналитическое решение модельной задачи
    2. Постановка разностной модельной задачи
    3. Реализация метода встречных прогонок
    4. Численное решение задачи с переменными коэффициентами
    


In [1]:
import numpy as np
from math import *

##### Описание переменных коэффициентов

In [2]:
def ka(x):
    return 1
def kb(x):
    return np.exp(np.cos(x))

In [3]:
def qa(x):
    return x**2
def qb(x):
    return x**2

In [4]:
def fa(x):
    return np.sin(x)
def fb(x):
    return np.sin(x)

In [5]:
x0 = 0
xL = 1
xbp = 1 / np.sqrt(3)
u0 = 1
u1 = 1

In [6]:
N = 11

In [7]:
L = N - 1
h = (xL - x0) / L
x = np.zeros(N)
for l in range(N):
    x[l] = x0 + l * h
    if x[l] < xbp and np.abs(x[l] - xbp) <= h:
        la = l
        lb = l + 1

##### Аналитическое решение модельной задачи

In [8]:
k_a = ka(xbp)
k_b = kb(xbp)
q_a = qa(xbp)
q_b = qb(xbp)
f_a = fa(xbp)
f_b = fb(xbp)

In [9]:
lambda_a = np.sqrt(q_a / k_a)
lambda_b = np.sqrt(q_b / k_b)

In [10]:
mu_a = f_a / q_a
mu_b = f_b / q_b

In [11]:
A11 = exp(-lambda_a * xbp) - exp(lambda_a * xbp)
A12 = exp(lambda_b * (2 - xbp)) - exp(lambda_b * xbp)
A21 = k_a * lambda_a * (exp(lambda_a * xbp) + exp(-lambda_a * xbp))
A22 = k_b * lambda_b * (exp(lambda_b * (2 - xbp)) + exp(lambda_b * xbp))

In [12]:
B1 = mu_b - mu_a + (mu_a - u0) * exp(lambda_a * xbp) - (mu_b - u1) * exp(lambda_b * (1 - xbp))
B2 = k_a * lambda_a * (u0 - mu_a) * exp(lambda_a * xbp) + k_b * lambda_b * (u1 - mu_b) * exp(lambda_b * (1 - xbp))

In [13]:
C1 = (((u0 - mu_a) * A11 - B1) * A22 - ((u0 - mu_a) * A21 - B2) * A12) / (A11 * A22 - A12 * A21)
C2 = (B1 * A22 - B2 * A12) / (A11 * A22 - A12 * A21)
C3 = (B2 * A11 - B1 * A21) / (A11 * A22 - A12 * A21)
C4 = (u1 - mu_b) * exp(lambda_b) - C3 * exp(2 * lambda_b)

In [14]:
u_0 = np.zeros(N)
for l in range(N):
    if l <= la:
        u_0[l] = C1 * exp(lambda_a * x[l]) + C2 * exp(-lambda_a * x[l]) + mu_a
    if l >= lb:
        u_0[l] = C3 * exp(lambda_b * x[l]) + C4 * exp(-lambda_b * x[l]) + mu_b

##### Численное решение модельной задачи

In [15]:
a = np.zeros(N)
b = np.zeros(N)
c = np.zeros(N)
d = np.zeros(N)

In [16]:
for l in range(1, L):
    if l <= la - 1:
        a[l] = k_a
        b[l] = -2 * k_a - q_a * h**2
        c[l] = k_a
        d[l] = -f_a * h**2
    if l >= lb + 1:
        a[l] = k_b
        b[l] = -2 * k_b - q_b * h**2
        c[l] = k_b
        d[l] = -f_b * h**2
        
a[0] = 0; a[la] = 0; a[lb] = 0; a[L] = 0
b[0] = 0; b[la] = 0; b[lb] = 0; b[L] = 0
c[0] = 0; c[la] = 0; c[lb] = 0; c[L] = 0
d[0] = 0; d[la] = 0; d[lb] = 0; d[L] = 0        

In [17]:
alpha = np.zeros(N)
beta = np.zeros(N)

In [18]:
for l in range(1, la):
    if l == 1:
        alpha[1] = -a[1] / b[1]
        beta[1] = (d[1] - c[1] * u0) / b[1]
    else:
        alpha[l] = -a[l] / (b[l] + c[l] * alpha[l - 1])
        beta[l] = (d[l] - c[l] * beta[l - 1]) / (b[l] + c[l] * alpha[l - 1])

for l in range(L-1, lb, -1):
    if l == L - 1:
        alpha[L - 1] = -c[L - 1] / b[L - 1]
        beta[L - 1] = (d[L - 1] - a[L - 1] * u1) / b[L - 1]
    else:
        alpha[l] = -c[l] / (b[l] + a[l] * alpha[l + 1])
        beta[l] = (d[l] - a[l] * beta[l + 1]) / (b[l] + a[l] * alpha[l + 1])
        
alpha[0] = 0; alpha[la] = 0; alpha[lb] = 0; alpha[L] = 0
beta[0] = 0; beta[la] = 0; beta[lb] = 0; beta[L] = 0

In [19]:
u_1 = np.zeros(N)

In [20]:
for l in range(la, 0, -1):
    if l == la:
        u_1[l] = (k_a * beta[la - 1] + k_b * beta[lb + 1]) / (k_a * (1 - alpha[la - 1]) + k_b * (1 - alpha[lb + 1]))
    else:
        u_1[l] = alpha[l] * u_1[l + 1] + beta[l]
        
for l in range(lb, L):
    if l == lb:
        u_1[l] = (k_a * beta[la - 1] + k_b * beta[lb + 1]) / (k_a * (1 - alpha[la - 1]) + k_b * (1 - alpha[lb + 1]))
    else:
        u_1[l] = alpha[l] * u_1[l - 1] + beta[l]
        
u_1[0] = u0
u_1[L] = u1

##### Численное решение задачи с переменными коэффициентами

In [21]:
k_minus = np.zeros(N)
k_plus = np.zeros(N)
q = np.zeros(N)
f = np.zeros(N)

In [22]:
for l in range(1, L):
    if l <= la - 1:
        k_minus[l] = ka(x[l] - h / 2)
        k_plus[l] = ka(x[l] + h / 2)
        q[l] = qa(x[l])
        f[l] = fa(x[l])
    if l >= lb + 1:
        k_minus[l] = kb(x[l] - h / 2)
        k_plus[l] = kb(x[l] + h / 2)
        q[l] = qb(x[l])
        f[l] = fb(x[l])
        
k_minus[0] = 0; k_minus[la] = 0; k_minus[lb] = 0; k_minus[L] = 0
k_plus[0] = 0; k_plus[la] = 0; k_plus[lb] = 0; k_plus[L] = 0
q[0] = 0; q[la] = 0; q[lb] = 0; q[L] = 0
f[0] = 0; f[la] = 0; f[lb] = 0; f[L] = 0

In [23]:
for l in range(1, L):
    if l <= la - 1:
        a[l] = k_plus[l]
        b[l] = -(k_plus[l] + k_minus[l] + q[l] * h**2)
        c[l] = k_minus[l]
        d[l] = -f[l] * h**2
    if l >= lb + 1:
        a[l] = k_plus[l]
        b[l] = -(k_plus[l] + k_minus[l] + q[l] * h**2)
        c[l] = k_minus[l]
        d[l] = -f[l] * h**2
        
a[0] = 0; a[la] = 0; a[lb] = 0; a[L] = 0
b[0] = 0; b[la] = 0; b[lb] = 0; b[L] = 0
c[0] = 0; c[la] = 0; c[lb] = 0; c[L] = 0
d[0] = 0; d[la] = 0; d[lb] = 0; d[L] = 0

In [24]:
for l in range(1, la):
    if l == 1:
        alpha[1] = -a[1] / b[1]
        beta[1] = (d[1] - c[1] * u0) / b[1]
    else:
        alpha[l] = -a[l] / (b[l] + c[l] * alpha[l - 1])
        beta[l] = (d[l] - c[l] * beta[l - 1]) / (b[l] + c[l] * alpha[l - 1])
        
for l in range(L-1, lb, -1):
    if l == L-1:
        alpha[L - 1] = -c[L - 1] / b[L - 1]
        beta[L - 1] = (d[L - 1] - a[L - 1] * u1) / b[L - 1]
    else:
        alpha[l] = -c[l] / (b[l] + a[l] * alpha[l + 1])
        beta[l] = (d[l] - a[l] * beta[l + 1]) / (b[l] + a[l] * alpha[l + 1])
        
alpha[0] = 0; alpha[la] = 0; alpha[lb] = 0; alpha[L] = 0
beta[0] = 0; beta[la] = 0; beta[lb] = 0; beta[L] = 0

In [25]:
u_2 = np.zeros(N)

In [26]:
for l in range(la, 0, -1):
    if l == la:
        u_2[l] = (k_a * beta[la - 1] + k_b * beta[lb + 1]) / (k_a * (1 - alpha[la - 1]) + k_b * (1 - alpha[lb + 1]))
    else:
        u_2[l] = alpha[l] * u_2[l + 1] + beta[l]
        
for l in range(lb, L):
    if l == lb:
        u_2[l] = (k_a * beta[la - 1] + k_b * beta[lb + 1]) / (k_a * (1 - alpha[la - 1]) + k_b * (1 - alpha[lb + 1]))
    else:
        u_2[l] = alpha[l] * u_2[l - 1] + beta[l]
        
u_2[0] = u0
u_2[L] = u1

In [27]:
x0_out = x0
xL_out = xL
N_out = 11
L_out = N_out - 1
h_out = (xL_out - x0_out) / L

In [28]:
x_out = np.zeros(N)

In [29]:
for l_out in range(0, L_out+1):
    x_out[l_out] = x0_out + l_out * h_out

In [30]:
diff = np.zeros(N)

##### Расчет погрешности

In [31]:
for l_out in range(0, L_out+1):
    for l in range(L+1):
        diff[l] = abs(u_0[l] - u_1[l])

In [32]:
import pandas as pd

In [33]:
pd.options.display.float_format ='{:,.6e}'.format

In [34]:
out_df = pd.DataFrame({
    'x': pd.Series(x),
    'u_0': pd.Series(u_0),
    'u_1': pd.Series(u_1),
    'diff': pd.Series(diff),
    'u_2': pd.Series(u_2)
})

In [35]:
out_df

,x,u_0,u_1,diff,u_2
0,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
1,1.000000e-01,1.007475e+00,1.006074e+00,1.400951e-03,1.004449e+00
2,2.000000e-01,1.012849e+00,1.010043e+00,2.805989e-03,1.008000e+00
3,3.000000e-01,1.016141e+00,1.011921e+00,4.219802e-03,1.009967e+00
4,4.000000e-01,1.017361e+00,1.011714e+00,5.647106e-03,1.009888e+00
5,5.000000e-01,1.016514e+00,1.009422e+00,7.092659e-03,1.007531e+00
6,6.000000e-01,1.014075e+00,1.009422e+00,4.653397e-03,1.007531e+00
7,7.000000e-01,1.011914e+00,1.008430e+00,3.484352e-03,1.006511e+00
8,8.000000e-01,1.008851e+00,1.006531e+00,2.320223e-03,1.004697e+00
9,9.000000e-01,1.004882e+00,1.003722e+00,1.159330e-03,1.002364e+00


$ u_0 $ - аналитическое решение модельной задачи

$ u_1 $ - численное решение модельной задачи

$ diff $ - погрешность

$u_2$ -- численное решение задачи с переменными коэффициентами
